Syntax natürlicher Sprachen, WS 2023/24

# 10 - Übung

In [2]:
import nltk

## Aufgabe 1 - Gewichte und Ableitungswahrscheinlichkeit

#### Betrachten Sie folgende (unvollständige) PCFG.

#### Ergänzen Sie die fehlenden (mit ** markierten) Gewichte und geben Sie die Berechnung für die Ableitungswahrscheinlichkeit durch den Viterbi-Parser an.

In [ ]:
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [**]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [**]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [**]
    NP   -> 'Jack'             [0.2]
""")

sent = "Jack saw telescopes"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)

In [ ]:
# Berechnung der Ableitungswahrscheinlichkeit:


## Aufgabe 2  - PCFG Parsing
#### Betrachten Sie das folgende PCFG-Parsing:

In [3]:
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [1]
    PP -> P NP [1]
    NP -> DET N [0.5]
    NP -> PRON [0.3]
    NP -> NP PP [0.2]
    VP -> V [0.1]
    VP -> V NP [0.3]
    VP -> VP PP [0.6]

    PRON -> "Er" [1]
    N -> "Mädchen" [0.5] | "Fernglas" [0.5]
    V -> "beobachtet" [1]
    DET -> "das" [0.5] | "dem" [0.5]
    P -> "mit" [1]
""")

sent = "Er beobachtet das Mädchen mit dem Fernglas"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (PRON Er))
  (VP
    (VP (V beobachtet) (NP (DET das) (N Mädchen)))
    (PP (P mit) (NP (DET dem) (N Fernglas))))) (p=0.00084375)
         S                                          
 ┌───────┴─────────────────┐                         
 │                         VP                       
 │               ┌─────────┴─────────┐               
 │               VP                  PP             
 │       ┌───────┴───┐           ┌───┴───┐           
 NP      │           NP          │       NP         
 │       │       ┌───┴─────┐     │   ┌───┴─────┐     
PRON     V      DET        N     P  DET        N    
 │       │       │         │     │   │         │     
 Er  beobachtet das     Mädchen mit dem     Fernglas



### a) Berechnen Sie die Wahrscheinlichkeit für die Ableitung mit dem Viterbi-Parser.

In [ ]:
# Berechnung der Ableitungswahrscheinlichkeit:


### b) Warum findet folgender Parser mehr als eine Ableitung?

In [4]:
parser = nltk.InsideChartParser(grammar)
for tree in parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (PRON Er))
  (VP
    (VP (V beobachtet) (NP (DET das) (N Mädchen)))
    (PP (P mit) (NP (DET dem) (N Fernglas))))) (p=0.00084375)
         S                                          
 ┌───────┴─────────────────┐                         
 │                         VP                       
 │               ┌─────────┴─────────┐               
 │               VP                  PP             
 │       ┌───────┴───┐           ┌───┴───┐           
 NP      │           NP          │       NP         
 │       │       ┌───┴─────┐     │   ┌───┴─────┐     
PRON     V      DET        N     P  DET        N    
 │       │       │         │     │   │         │     
 Er  beobachtet das     Mädchen mit dem     Fernglas

(S
  (NP (PRON Er))
  (VP
    (V beobachtet)
    (NP
      (NP (DET das) (N Mädchen))
      (PP (P mit) (NP (DET dem) (N Fernglas)))))) (p=0.00028125)
         S                                          
 ┌───────┴─────────────────┐                         
 │            

In [ ]:
# Antwort:


### c) Vergleichen Sie das Tracing folgender probabilistischer Parser und erläutern Sie:

In [ ]:
inside_parser = nltk.InsideChartParser(grammar, trace=1)
for tree in inside_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

In [ ]:
longest_parser = nltk.LongestChartParser(grammar, trace=1)
for tree in longest_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

In [ ]:
viterbi_parser = nltk.ViterbiParser(grammar, trace=2)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

In [ ]:
beam_parser = nltk.InsideChartParser(grammar, beam_size=7, trace=1)
for tree in beam_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

## Aufgabe 3 - Abschätzung von Regelwahrscheinlichkeiten

###  a) Herunterladen von Ressourcen und Verarbeitung von CFG-Regeln mit NLTK

#### Laden Sie zunächst die Ressource „corpora/treebank“ mithilfe des NLTK-Download-Managers herunter, falls diese noch nicht installiert ist.

In [ ]:
# nltk.download()

 #### Hier ein Beispiel für geparste Sätze in der Penn Treebank:

In [5]:
from nltk.corpus import treebank

for tree in treebank.parsed_sents('wsj_0001.mrg'):
    print(tree)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))
(S
  (NP-SBJ (NNP Mr.) (NNP Vinken))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (NN chairman))
      (PP
        (IN of)
        (NP
          (NP (NNP Elsevier) (NNP N.V.))
          (, ,)
          (NP (DT the) (NNP Dutch) (VBG publishing) (NN group))))))
  (. .))


#### Hier ein Beispiel für die Verarbeitung solcher CFG-Regeln im NLTK:

In [6]:
productions = tree.productions()
productions

[S -> NP-SBJ VP .,
 NP-SBJ -> NNP NNP,
 NNP -> 'Mr.',
 NNP -> 'Vinken',
 VP -> VBZ NP-PRD,
 VBZ -> 'is',
 NP-PRD -> NP PP,
 NP -> NN,
 NN -> 'chairman',
 PP -> IN NP,
 IN -> 'of',
 NP -> NP , NP,
 NP -> NNP NNP,
 NNP -> 'Elsevier',
 NNP -> 'N.V.',
 , -> ',',
 NP -> DT NNP VBG NN,
 DT -> 'the',
 NNP -> 'Dutch',
 VBG -> 'publishing',
 NN -> 'group',
 . -> '.']

In [7]:
productions[0].lhs()

S

In [8]:
productions[0].rhs()

(NP-SBJ, VP, .)

In [9]:
type(productions[0].lhs())

nltk.grammar.Nonterminal

In [10]:
productions[0].lhs() == nltk.grammar.Nonterminal('S')

True

### b) von Daten zu Regelwahrscheinlichkeiten

#### Gegeben sei folgende kontextfreie Grammatik:

In [11]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP PP
VP -> V NP
NP -> DET N
NP -> NP PP
PP -> P NP

DET -> "the" | "a"
N -> "boy" | "woman" | "telescope"
V -> "saw"
P -> "with"
""")

#### Sie modelliert sehr einfache Sätze der Form `SBJ` *saw* `OBJ` mit optionaler Präpositionalphrase am Ende. Diese Präpositionalphrase kann entweder der näheren Bestimmung des Objekts oder der näheren Bestimmung der in der Verbalphrase ausgedrückten Handlung dienen.


In [12]:
sent = "the boy saw a woman with a telescope"

parser = nltk.ChartParser(grammar,trace=0)
for tree in parser.parse(sent.split()):
    tree.pretty_print(unicodelines=True)

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                     
     │       │                 NP                   
     │       │       ┌─────────┴────┐                
     │       │       │              PP              
     │       │       │         ┌────┴

#### Im folgenden sollen aus einer Treebank Wahrscheinlichkeiten für die einzelnen Regeln extrahiert werden, um diese Ambiguität aufzulösen.

#### Nutzen Sie das im NLTK enthaltene Sample der Penn Treebank (nach Installation unter `nltk.corpus.treebank` zu finden) zunächst zur Identifikation der für eine Disambiguierung nützlichen (Teil-)bäume der Penn Treebank. Dazu zählen Sie zu einem gegebenen Nonterminal als LHS die Produktionen in der Treebank.

#### *Hinweis:* Sie können sich bei der Analyse auf die 30 häufigsten Konstruktionen der Baumbank beschränken.


In [13]:
from collections import defaultdict

def find_relevant_constructions(lhs):
    counter = defaultdict(int)
    #TO_DO: zähle Produktionen in treebank mit lhs als linker Seite
    
    constructions = [ (k, counter[k]) for k in sorted(counter.keys(), key=counter.__getitem__, reverse=True) ]
    return constructions[:30]

find_relevant_constructions("S") #example

[]

 #### Für welche Regeln müssen wir die Wahrscheinlichkeiten berechnen, wenn wir mit statistischen Methoden untersuchen wollen, ob PPs häufiger Teil der VP oder Teil der NP sind?

In [ ]:
# Antwort:


### c) Abschätzung der Wahrscheinlichkeiten für relevante Regeln

#### Approximieren Sie mittels vergleichbarer Konstruktionen in der Penn Treebank die Wahrscheinlichkeiten der für die Disambiguierung der PP-Ambiguität relevanten Regeln mit Maximum Likelihood Estimation (MLE).

#### (i) Zählen Sie zunächst für die V+NP+PP-Konstruktion, wie oft sie in der Penn Treebank vorkommen und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten:

$$P(V, N\!P, P\!P \mid V\!P) = \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow \setminus*)}$$

**hier (vgl. Grammatik):**

$$= \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow V\:N\!P\:P\!P) + count(V\!P \rightarrow V\:N\!P)}$$

In [14]:
find_relevant_constructions('VP')

[]

In [ ]:
vp_v_np_pp_frq = #TO_DO
vp_v_np_without_frq = #TO_DO

vp_with_pp = vp_v_np_pp_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)
vp_without = vp_v_np_without_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)

(vp_with_pp, vp_without)

#### (ii) Zählen Sie anschließend, wie oft die NP+PP-Konstruktion in der Penn Treebank vorkommt und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten. Das Vorgehen wird in folgender Formel veranschaulicht:

$$P(N\!P, P\!P \mid N\!P) = \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \setminus*)}$$

**hier:**

$$= \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \:N\!P\:P\!P) + count(N\!P \rightarrow DET\:N)}$$



In [15]:
find_relevant_constructions('NP')

[]

In [ ]:
np_np_pp_frq = #TO_DO
np_n_without_frq = #TO_DO

np_with_pp = np_np_pp_frq / (np_np_pp_frq + np_n_without_frq)
np_without = np_n_without_frq / (np_np_pp_frq + np_n_without_frq)

(np_with_pp, np_without)

### d) Erstellen einer PCFG

#### Die aus den Daten extrahierten relativen Häufigkeiten sollen nun zur Erstellung einer probabilistischen kontextfreien Grammatik (PCFG)  genutzt werden.

In [ ]:
(vp_with_pp, vp_without, np_with_pp, np_without)

In [ ]:
pcfg = f"""
S -> NP VP     [1.0]
VP -> V NP PP  [{vp_with_pp}]
VP -> V NP     [{vp_without}]
NP -> DET N    [{np_without}]
NP -> NP PP    [{np_with_pp}]
PP -> P NP     [1.0]

DET -> "the"     [0.7]
DET -> "a"       [0.3]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
"""

grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)

#### Testen Sie Ihre so erstellte Grammatik nun, indem Sie folgenden Satz parsen:

- *the boy saw a woman with a telescope*

In [ ]:
sent = "the boy saw a woman with a telescope"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

#### Wenn Sie sich die extrahierten Wahrscheinlichkeiten und das disambiguierte Ergebnis ansehen, überrascht Sie dann das Ergebnis der Syntaxanalyse?

In [ ]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp
#P(VP->VP+NP+PP)

In [ ]:
# relevante Teilwahrscheinlichkeit NP-Attachment:
vp_without * np_with_pp
#P(VP->VP+NP) * P(NP->NP+PP)

In [ ]:
# Antwort:


### e) Vergleichen Sie dieses Ergebnis mit der PCFG-Analyse mit folgenden abweichenden Regelwahrscheinlichkeiten. 

#### Warum wird hier, obwohl weiter `vp_with_pp < np_with_pp` gilt, der VP-Attachment-Baum als der wahrscheinlichere ausgewählt? 

In [17]:
vp_with_pp = 0.2
vp_without = 0.8
np_with_pp = 0.22
np_without = 0.78

In [18]:
pcfg = f"""
S -> NP VP     [1.0]
VP -> V NP PP  [{vp_with_pp}]
VP -> V NP     [{vp_without}]
NP -> DET N    [{np_without}]
NP -> NP PP    [{np_with_pp}]
PP -> P NP     [1.0]

DET -> "the"     [0.7]
DET -> "a"       [0.3]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
"""

grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)

Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP PP [0.2]
    VP -> V NP [0.8]
    NP -> DET N [0.78]
    NP -> NP PP [0.22]
    PP -> P NP [1.0]
    DET -> 'the' [0.7]
    DET -> 'a' [0.3]
    N -> 'boy' [0.4]
    N -> 'woman' [0.4]
    N -> 'telescope' [0.2]
    V -> 'saw' [1.0]
    P -> 'with' [1.0]


In [19]:
sent = "the boy saw a woman with a telescope"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (DET the) (N boy))
  (VP
    (V saw)
    (NP (DET a) (N woman))
    (PP (P with) (NP (DET a) (N telescope))))) (p=0.000191339)
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope



In [20]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp
#P(VP->VP+NP+PP)

0.2

In [21]:
# relevante Teilwahrscheinlichkeit NP-Attachment:
vp_without * np_with_pp
#P(VP->VP+NP) * P(NP->NP+PP)

0.17600000000000002

In [ ]:
# Antwort:
